<a href="https://colab.research.google.com/github/zxfastxz/-bcb-pyspark/blob/main/Untitled4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
import json

In [2]:
url = (
    "https://olinda.bcb.gov.br/olinda/servico/MPV_DadosAbertos/versao/v1/"
    "odata/Quantidadeetransacoesdecartoes(trimestre=@trimestre)?@trimestre='2024'"
    "&$top=100&$format=json&$select=trimestre,nomeBandeira,"
    "qtdCartoesEmitidos,qtdCartoesAtivos,valorTransacoesNacionais")

In [3]:
def requisicao_api(link):
    resposta = requests.get(link)
    if resposta.status_code == 200:
        dados = resposta.json()
        with open("resultado.json", "w", encoding="utf-8") as f:
            json.dump(dados, f, ensure_ascii=False, indent=4)
        print("Dados obtidos da API com sucesso.")
        return dados["value"]
    else:
        raise Exception(f"Erro ao acessar API: {resposta.status_code}")

In [4]:
registros = requisicao_api(url)

Dados obtidos da API com sucesso.


In [5]:
registros[:3]

[{'trimestre': '20231',
  'nomeBandeira': 'VISA',
  'qtdCartoesEmitidos': 739765,
  'qtdCartoesAtivos': 404801,
  'valorTransacoesNacionais': 1269264800.22},
 {'trimestre': '20173',
  'nomeBandeira': 'MasterCard',
  'qtdCartoesEmitidos': 983369,
  'qtdCartoesAtivos': 771386,
  'valorTransacoesNacionais': 9648105300.53},
 {'trimestre': '20192',
  'nomeBandeira': 'MasterCard',
  'qtdCartoesEmitidos': 13454781,
  'qtdCartoesAtivos': 5296965,
  'valorTransacoesNacionais': 7045628606.32}]

In [6]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

In [9]:
!apt-get update -q
!apt-get install -y openjdk-11-jdk-headless

Get:1 https://cli.github.com/packages stable InRelease [3,917 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:5 https://cli.github.com/packages stable/main amd64 Packages [344 B]
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:7 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,421 kB]
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,123 kB]
Hit:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 P

In [10]:
!pip install -q pyspark==3.5.1

In [11]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["PATH"] += os.pathsep + os.path.join(os.environ["JAVA_HOME"], "bin")

In [12]:
print("Java e PySpark configurados com sucesso!")

Java e PySpark configurados com sucesso!


In [13]:
spark = SparkSession.builder \
    .appName("BCB_Cartoes_PySpark") \
    .master("local[*]") \
    .getOrCreate()

In [14]:
df = spark.createDataFrame(registros)

In [15]:
df.printSchema()
df.show(5, truncate=False)

root
 |-- nomeBandeira: string (nullable = true)
 |-- qtdCartoesAtivos: long (nullable = true)
 |-- qtdCartoesEmitidos: long (nullable = true)
 |-- trimestre: string (nullable = true)
 |-- valorTransacoesNacionais: double (nullable = true)

+------------+----------------+------------------+---------+------------------------+
|nomeBandeira|qtdCartoesAtivos|qtdCartoesEmitidos|trimestre|valorTransacoesNacionais|
+------------+----------------+------------------+---------+------------------------+
|VISA        |404801          |739765            |20231    |1.26926480022E9         |
|MasterCard  |771386          |983369            |20173    |9.64810530053E9         |
|MasterCard  |5296965         |13454781          |20192    |7.04562860632E9         |
|VISA        |525736          |1017551           |20171    |1.36791442416E9         |
|Elo         |11255           |28306             |20242    |1.0947659622E8          |
+------------+----------------+------------------+---------+-----------

In [16]:
df_sel = df.select("trimestre", "nomeBandeira", "qtdCartoesEmitidos", "qtdCartoesAtivos", "valorTransacoesNacionais")

In [17]:
df_taxa = df_sel.withColumn("taxa_uso", (F.col("qtdCartoesAtivos") / F.col("qtdCartoesEmitidos")) * 100)


In [18]:
df_filtrado = df_taxa.filter(F.col("qtdCartoesEmitidos") > 1_000_000)

In [19]:
df_ordenado = df_filtrado.orderBy(F.col("valorTransacoesNacionais").desc())

In [20]:
df_final = df_ordenado.select(
    F.col("trimestre").alias("Trimestre"),
    F.col("nomeBandeira").alias("Bandeira"),
    F.col("qtdCartoesEmitidos").alias("Cartoes_Emitidos"),
    F.col("qtdCartoesAtivos").alias("Cartoes_Ativos"),
    F.col("valorTransacoesNacionais").alias("Valor_Transacoes"),
    F.round(F.col("taxa_uso"), 2).alias("Taxa_Uso_Perc")
)

In [21]:
df_final.show(10, truncate=False)

+---------+----------+----------------+--------------+-----------------+-------------+
|Trimestre|Bandeira  |Cartoes_Emitidos|Cartoes_Ativos|Valor_Transacoes |Taxa_Uso_Perc|
+---------+----------+----------------+--------------+-----------------+-------------+
|20231    |MasterCard|81001188        |49472663      |9.705630688741E10|61.08        |
|20233    |MasterCard|17675563        |12563150      |5.785715789661E10|71.08        |
|20232    |MasterCard|4836618         |3596464       |5.450269382557E10|74.36        |
|20221    |MasterCard|13267307        |10663542      |4.629259282228E10|80.37        |
|20222    |VISA      |25023382        |16262619      |4.039281547214E10|64.99        |
|20222    |MasterCard|41085473        |15303441      |3.557055092734E10|37.25        |
|20124    |VISA      |142130307       |37797073      |3.16555754859E10 |26.59        |
|20251    |MasterCard|9123292         |3733708       |2.117302777338E10|40.93        |
|20244    |VISA      |19972671        |1780

In [22]:
df_final.show(5)

+---------+----------+----------------+--------------+-----------------+-------------+
|Trimestre|  Bandeira|Cartoes_Emitidos|Cartoes_Ativos| Valor_Transacoes|Taxa_Uso_Perc|
+---------+----------+----------------+--------------+-----------------+-------------+
|    20231|MasterCard|        81001188|      49472663|9.705630688741E10|        61.08|
|    20233|MasterCard|        17675563|      12563150|5.785715789661E10|        71.08|
|    20232|MasterCard|         4836618|       3596464|5.450269382557E10|        74.36|
|    20221|MasterCard|        13267307|      10663542|4.629259282228E10|        80.37|
|    20222|      VISA|        25023382|      16262619|4.039281547214E10|        64.99|
+---------+----------+----------------+--------------+-----------------+-------------+
only showing top 5 rows



In [23]:
print("Total de registros:", df_final.count())

Total de registros: 46


In [24]:
df_final.describe().show()

+-------+------------------+----------------+--------------------+-----------------+--------------------+-----------------+
|summary|         Trimestre|        Bandeira|    Cartoes_Emitidos|   Cartoes_Ativos|    Valor_Transacoes|    Taxa_Uso_Perc|
+-------+------------------+----------------+--------------------+-----------------+--------------------+-----------------+
|  count|                46|              46|                  46|               46|                  46|               46|
|   mean|20180.217391304348|            NULL|1.3011132304347826E7|5950491.521739131|1.289888710093522...|48.53086956521738|
| stddev| 45.05153249013936|            NULL|2.3588275958315555E7|9304027.780015064|1.937759795594402E10|23.72024384604724|
|    min|             20111|Bandeira própria|             1017551|            69204|       1.817287784E7|             3.86|
|    max|             20251|            VISA|           142130307|         49472663|   9.705630688741E10|            96.82|
+-------

In [25]:
df_final.select(F.mean("Valor_Transacoes").alias("Media_Valor_Transacoes")).show()

+----------------------+
|Media_Valor_Transacoes|
+----------------------+
|  1.289888710093522...|
+----------------------+



In [26]:
df_final.groupBy("Bandeira").agg(
    F.sum("Valor_Transacoes").alias("Soma_Transacoes"),
    F.avg("Taxa_Uso_Perc").alias("Media_Taxa_Uso")
).show()

+----------------+--------------------+------------------+
|        Bandeira|     Soma_Transacoes|    Media_Taxa_Uso|
+----------------+--------------------+------------------+
|Bandeira própria|       1.145848954E9|             96.82|
|            VISA|  1.6473209573355E11| 51.15764705882352|
|             Elo|2.3080790169100003E9|23.354999999999997|
|      MasterCard|  4.2514461006072E11|           48.5188|
|          Outras|       1.817287784E7|              6.24|
+----------------+--------------------+------------------+

